# Deutsch's Algorithm
> First I load some helper functions, to help me measure/normalize the qubits. I explain these more in depth in my "Teleportation" Algorithm on my github.

In [1]:
"""
Author: Jack Waslen
Date: April 2024
"""

three_bits = ["000","001","010","011","100","101","110","111"]
two_bits = ["00","01","10","11"]
one_bits = ["0","1"]


def measureFirstQubit(circuit_state):
    qubit_factors = circuit_state
    factors = absolute_square(qubit_factors)
    outcomes = {"0":0,"1":0}
    
    i = 0
    for value in factors.values():
        if i < 4:
            outcomes["0"] += round(value,3)
        else:
            outcomes["1"] += round(value,3)
        i += 1
    return normalization(outcomes)


def normalization(probabilities):
    N = 0

    possible_values = three_bits
    if len(probabilities) == 4:
        possible_values = two_bits
    if len(probabilities) == 2:
        possible_values = one_bits
        
    for combo in possible_values:
        N += probabilities[combo]

    if N != 1:
        for combo in possible_values:
            probabilities[combo] = probabilities[combo] / N
    return probabilities
                
def absolute_square(factors):
    absolute_squares = {}
    
    # Adjusting function based on if were calculating one bit, two bits or three bits.
    bit_combinations = three_bits
    
    if len(factors) == 2:
        bit_combinations = one_bits
    if len(factors) == 4:
        bit_combinations = two_bits
    
    # Loop through mapping probabilities to bit values
    for index in range(len(factors)):
        probability_amplitude = abs(factors[index].real) ** 2 + abs(factors[index].imag) ** 2
        absolute_squares[bit_combinations[index]] = round(probability_amplitude,3)

    return absolute_squares


> This measure Input Qubit takes the four factors (alpha, beta, gamma, delta) and measures only the first qubit, whether is it a zero or a one.

In [2]:
def measureInputQubit(circuit):
    zero = 0
    one = 0
    
    # qubit_factors = [alpha, beta, gamma, delta]
    qubit_factors = circuit.state
    
    # factors is the corresponding probabilities
    factors = normalization(absolute_square(qubit_factors))
    index = 0
    
    # Add the corresponding probabilities for input qubit = 1 
    # and input qubit = 0
    for (key, value) in factors.items():
        if index <= 1:
            zero += value
        if index > 1:
            one += value
        index += 1
        
    if zero != 1 and one != 1:
        print("Catastrophic Error!")
    if zero == 1:
        print("We measure Zero! The function is Balanced.")

    if one == 1:
        print("We measure One! The function is Constant.")


> This function performs deutsch's algorithm. I use a identity gate and a CNOT to model a balanced function vs a constant function respectively.

In [3]:
import numpy as np
from math import sqrt
import dwave.gate.operations as gate
from dwave.gate.simulator import simulate
from dwave.gate import Circuit
from dwave.gate.registers.registers import ClassicalRegister, QuantumRegister
from math import pi

def deutsch_algorithm(balancedOracle):
    # Circuit has two qubits, zero classical bits
    circuit = Circuit(2,0)

    # Entering the circuit
    with circuit.context as (qubit, cbits):
        # First Qubit
        input_qubit = qubit[0]

        # Last Qubit, ie. second qubit
        output_qubit = qubit[-1]

        # Apply the required gates
        gate.X(input_qubit)
        gate.X(output_qubit)

        gate.Hadamard(input_qubit)
        gate.Hadamard(output_qubit)


        # Apply Oracle
        
        if balancedOracle:
            # This CNOT represents a balanced function
            gate.CNOT(control=input_qubit, target=output_qubit)
        else:
            # This identity represents a constant function
            gate.Identity(input_qubit)
        
        # Finish Protocol
        gate.Hadamard(input_qubit)

        simulate(circuit)

        measureInputQubit(circuit)


In [4]:
print("Our Data Matches with a Balanced Function: ")
print("Running Deutsch's ...")
deutsch_algorithm(balancedOracle=True)

print()
print("Our Data Matches with a Constant Function: ")
print("Running Deutsch's ...")
deutsch_algorithm(balancedOracle=False)

Our Data Matches with a Balanced Function: 
Running Deutsch's ...
We measure Zero! The function is Balanced.

Our Data Matches with a Constant Function: 
Running Deutsch's ...
We measure One! The function is Constant.
